<a href="https://colab.research.google.com/github/robertoir/robotica_adaptativa_collab/blob/master/roboticaadaptativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://gym.openai.com/docs/

https://gym.openai.com/

Frozen-Lake, simula una pista de patinaje sobre hielo. Esta pista está dividida en 16 celdas (4x4), y en algunas de ellas se ha roto el hielo (H). El patinador empieza a patinar en la posición superior izuqierda y us objetivo es alcanzar la posición inferior derecha evitando caer en algún agujero). Si cae en agujero el episodio termina y la recompesa (reward) será cero. Si llega a la celda destino la recompensa será uno y termina el episodio. Al ser 4x4 hay 16 estados se numeran del 0 al 15. Hay un espacio de acciones compuesto de movimientos en cuatro direcciones (left, down, right, up). También hay valla entorno a la pista, si el agente intenta salir de la cuadrícula, robta y queda en la misma celda desde la que lo intentó. Existe un variable (slippery) que permite introducir incertidumbre en el problema. Si el patinador quiere ir hacia una dirección, hay 1/3 de probablidad de que lo logre (33%), 66% que vaya en otra dirección

In [ ]:
import gym

env=gym.make("FrozenLake-v0",is_slippery=False)
env.reset()
env.render()
is_done=False
t=0

while not is_done:
  accion=env.action_space.sample()
  estado,refuerzo,is_done,_=env.step(accion)
  env.render()
  t=t+1

print ("ultimo estado ",estado)
print ("ultimo refuerzo ",refuerzo)
print ("pasos_ejecutados ",t)



Este código que acabamos de ver, no trabaja con agentes. Reset reinicia la partida, redner muestra el estado del tablero (sistema en este caso), sample escoge una acción de forma aleatoria, step la ejecuta  y devuelve las consecuencias.

In [ ]:
import gym

class Agent:
  
  def __init__(self):
    self.internalenv=gym.make("FrozenLake-v0",is_slippery=False)

  def select_action(self):
    return self.internalenv.action_space.sample()

env=gym.make("FrozenLake-v0",is_slippery=False)
env.reset()
env.render()
is_done=False
t=0

agent=Agent()

while not is_done:
  accion=env.action_space.sample()
  estado,refuerzo,is_done,_=env.step(accion)
  env.render()
  t=t+1

print ("ultimo estado ",estado)
print ("ultimo refuerzo ",refuerzo)
print ("pasos_ejecutados ",t)

In [ ]:
env=gym.make("FrozenLake-v0",is_slippery=False)
print ("espacio de estados ",env.observation_space)
#Discrete (16) indica que son 16 estados posibles, desde el 0 al 15
#Hay un estado inicial, el cero y 5 estados terminales
print ("mostramos el espacio de estados")
print (env.action_space)
#Dispone de un espacio de estados discreto, formado por 4 acciones posibles
#izquierda, abajo,derecha y arriba

env.reset()
print (env.P)

nS=env.observation_space.n
state=0

for next_state in env.P[state][0]:
  print ("info del siguiente estado")
  print (next_state)
  probability = next_state[0]
  new_state = next_state[1]
  reward = next_state[2]
  print ("nuevo estado ",new_state," al que transita con probabilidad ",probability," y recibe el refuerzo ",reward)


espacio de estados  Discrete(16)
mostramos el espacio de estados
Discrete(4)
{0: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 4, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}, 1: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 5, 0.0, True)], 2: [(1.0, 2, 0.0, False)], 3: [(1.0, 1, 0.0, False)]}, 2: {0: [(1.0, 1, 0.0, False)], 1: [(1.0, 6, 0.0, False)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 2, 0.0, False)]}, 3: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 7, 0.0, True)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 3, 0.0, False)]}, 4: {0: [(1.0, 4, 0.0, False)], 1: [(1.0, 8, 0.0, False)], 2: [(1.0, 5, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}, 5: {0: [(1.0, 5, 0, True)], 1: [(1.0, 5, 0, True)], 2: [(1.0, 5, 0, True)], 3: [(1.0, 5, 0, True)]}, 6: {0: [(1.0, 5, 0.0, True)], 1: [(1.0, 10, 0.0, False)], 2: [(1.0, 7, 0.0, True)], 3: [(1.0, 2, 0.0, False)]}, 7: {0: [(1.0, 7, 0, True)], 1: [(1.0, 7, 0, True)], 2: [(1.0, 7, 0, True)], 3: [(1.0, 7, 0, True)]}, 8: {0: [(1.0, 8, 0.0, False)], 1: [(1.0, 12, 0

In [ ]:
#input("Presione una tecla para continuar...")
#Muestra {estado: {accion:[(Pss',s',rss',terminal)]}}
env=gym.make("FrozenLake-v0",is_slippery=True)
env.reset()
env.render()

env.P
nS=env.observation_space.n
state=10
action=1

for next_state in env.P[state][action]:
  print ("info del siguiente estado")
  print (next_state)
  probability = next_state[0]
  new_state = next_state[1]
  reward = next_state[2]
  print ("nuevo estado ",new_state," al que transita con probabilidad ",probability," y recibe el refuerzo ",reward)




In [ ]:
import gym
import numpy as np
import random
import time

GAMMA = 0.9

class Agent:
  politica_cambiada=True

  def __init__(self):
    print ("Inicializamos la valoración de los estados", env.observation_space.n)
    #solo se podrían inicializar los estados no terminales con un valor distinto de cero..
    self.value_function = np.zeros(env.observation_space.n)
    #construimos una política determininsta, en cada estado se propone una única acción (las acciones son valores enterores entre 0 y 3)
    #la politica inicial sugiere ejecutar la acción cero para todos los estados
    self.policy = np.zeros(env.observation_space.n, dtype=int)
    


  def policy_evaluation(self):
    #P, nS, nA, policy, gamma=0.9, tol=1e-3

    umbral_theta=1e-3
    
    self.value_function = np.zeros(env.observation_space.n)
    copia_value_function =  np.zeros(env.observation_space.n, dtype=int)
    nS=env.observation_space.n
    
    while True:
        for state in range(nS):
            #asumimos política determinista, por lo que el primer sumatorio del algoritmo desaparece...
            for next_state in env.P[state][self.policy[state]]:
                probability = next_state[0]
                new_state = next_state[1]
                reward = next_state[2]
                #print(gamma*probability*value_function[new_state])
                copia_value_function[state] += probability*(reward + GAMMA*self.value_function[new_state])
            #print ("estado ",state," valoración original ",self.value_function[state],"nueva valoracion ",copia_value_function[state])
                
        diff = max(np.abs(copia_value_function - self.value_function))
        #print ("delta ",diff)
        #input ("pulsa para seguir ")
        if diff <= umbral_theta:
            self.value_function = copia_value_function.copy()
            break
        else:
            self.value_function = copia_value_function.copy()
            #reiniciamos para el sumatorio de la siguiente interaccion
            copia_value_function = np.zeros(nS) 
            #print(updated_value_function, '\n')
            #print(value_function)

  
  
  def policy_improvement(self):
     #P, nS, nA, value_from_policy, policy, gamma=0.9
     #definimos el numero de estados y el  numero de acciones...
     nS=env.observation_space.n
     nA=env.action_space.n
     #vamos a construir una nueva politica determinista
     self.politica_cambiada=False    
     ############################
     cambios=1
     while (cambios>0):
       cambios=0
       for state in range(nS):
        valoraciones = []
        candidatas=[]
        old_action=self.policy[state]
        for action in range(nA):
            value = 0
            for next_state in env.P[state][action]:
                probability = next_state[0]
                new_state = next_state[1]
                reward = next_state[2]
                value += probability*(reward + GAMMA*self.value_function[new_state])
            valoraciones.append(value)
            #print ("accion ",action,"valoracion ",value)
            
        #candidatas=actions.index(np.max(actions)) 
        #np.asarray -->convierte la tupla que devuelve np.where en un array (pero tiene forma de matriz)
        #flatten --> convierte la matriz en vector, uniendo por filas
        candidatas=np.asarray(np.where(max(valoraciones) == valoraciones)).flatten()
        #print ("valoraciones ",actions)
        #print ("acciones candidatas entre las que escogeremos al azar ",candidatas)       

        self.policy[state]=random.choice(candidatas)
        #self.policy[state] = np.argmax(actions)
        #print ("para el estado ",state,"sugerimos ahora la accion ",self.policy[state]) 
        if (np.abs(valoraciones[old_action]-valoraciones[self.policy[state]])>0.001):
          #print ("old action ",old_action,"valoracion ",valoraciones[old_action],"nueva accion ",self.policy[state],"valoracion nueva ",valoraciones[self.policy[state]])
          cambios=cambios+1
          self.politica_cambiada=True
        #print ("cambios hechos ",cambios)

def train(agent): 

    #best_reward = 0.0 
    #while best_reward < REWARD_THRESHOLD:
    while (agent.politica_cambiada):
      agent.policy_evaluation()
      print ("termine la valoración de la política, ahora la mejoramos....")
      agent.policy_improvement()
      print ("termine policy improvement vemos si procede un nuevo ciclo ",agent.politica_cambiada)
      #input ("pulsa para un nuevo ciclo...")


def ejecuta_episodio(agent,ciclos_maximos):
  ciclos_ejecutados=0
  episode_reward = 0
  estado = env.reset()
  for t in range(ciclos_maximos):
    env.render()
    time.sleep(0.25)
    accion = agent.policy[estado]
    estado, refuerzo, acabado, _ = env.step(accion)
    episode_reward += refuerzo
    ciclos_ejecutados += 1
    if acabado:
      break
  
  env.render()
  if not acabado:
    print("El agente no ha sido capaz de finalizar en  un máximo de {} ciclos.".format(ciclos_maximos))
  else:
    print("el agente ha finalizado en %d diclos " % ciclos_ejecutados)
    print("el refuerzo acumulado durante el episodio ha sido %f" % episode_reward)


#"Deterministic-4x4-FrozenLake-v0"
#env=gym.make("FrozenLake-v0",map_name='4x4',is_slippery=False)

#"Deterministic-8x8-FrozenLake-v0"
#env=gym.make("FrozenLake-v0",map_name='8x8',is_slippery=False)

#"Stochastic-4x4-FrozenLake-v0"
#env=gym.make("FrozenLake-v0",map_name='4x4',is_slippery=True)


print ("espacio de estados ",env.observation_space)
print ("el numero de estados es", env.observation_space.n)
#Discrete (16) indica que son 16 estados posibles, desde el 0 al 15
#Hay un estado inicial, el cero y 5 estados terminales
print ("mostramos el espacio de estados")
print (env.action_space)
#Dispone de un espacio de estados discreto, formado por 4 acciones posibles
#izquierda, abajo,derecha y arriba

env.reset()
env.render()
input ("pulsa para seguir ")
#env.P

agent = Agent()
train(agent)
ejecuta_episodio(agent,20)



Esto muestra la matriz con las probabilidades de transición (función de transferecia). Cada acción incluye una lista, donde cada elemento es una tupla que muestra la probabilidad de pasar al siguietne estado, el siguiente estado, la recompensa y si el episodio termina allí o no

In [ ]:
import gym

env=gym.make("FrozenLake-v0",is_slippery=True)



Vemos en este caso la función de transición. Cada acción incluye una lista, donde cada elemento es una tupla que muestra la probabilidad de pasar al siguietne estado, el siguiente estado, la recompensa y si el episodio termina allí o no

In [ ]:
env.P

ESta función de transición implementa que hay solo un tercio (33.33%) de probabilidad de que vayamos al etado de la celda prevista, y dos tercios (66.66%) de probabilidad de que vayamos en direcciones orgonales no deseadas